<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

# Инструкция по выполнению проекта

- Загрузите и изучите данные.
- Ответьте на вопрос и обоснуйте решение. 

    Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
    1. Изменится. Приведите примеры матриц.
    2. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.
- Предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.
- Запрограммируйте этот алгоритм, применив матричные операции. Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.

# Описание данных

- Набор данных находится в файле `/datasets/insurance.csv`. 
- **Признаки:** пол, возраст и зарплата застрахованного, количество членов его семьи.
- **Целевой признак:** количество страховых выплат клиенту за последние 5 лет.


## Загрузка данных

In [40]:
import os
import pandas as pd
import numpy as np 

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

RANDOM_STATE = 12345


In [41]:
pth1 = '/Users/ramilvaleev/Desktop/Yandex Projects/Проекты в работе/Защита персональных данных/insurance.csv'
pth2 = '/datasets/insurance.csv'

if os.path.exists(pth2):
    data = pd.read_csv(pth2)
elif os.path.exists(pth1):
    data = pd.read_csv(pth1)
else:
    print('Something is wrong')

In [42]:
def check_df(df):
    rows_count, columns_count = df.shape
    print('Количество стобцов:', columns_count)
    print('Количество строк:', rows_count)
    
    zeros_age = df[df['Возраст'] == 0]['Возраст'].count()
    percent_age = zeros_age / df.shape[0]
    print('Количество нулевых значений(Возраст) -', zeros_age)
    print('Процент нулевых значений(Возраст) - {:.1%}'.format(percent_age))
    
    zeros_salary = df[df['Зарплата'] == 0]['Зарплата'].count()
    percent_salary = zeros_salary / df.shape[0]
    print('Количество нулевых значений(Зарплата) -', zeros_salary)
    print('Процент нулевых значений(Зарплата) - {:.1%}'.format(percent_salary))
    
    display(df.head(10))
    display(df.describe())
    df.info()

In [43]:
check_df(data)

Количество стобцов: 5
Количество строк: 5000
Количество нулевых значений(Возраст) - 0
Процент нулевых значений(Возраст) - 0.0%
Количество нулевых значений(Зарплата) - 0
Процент нулевых значений(Зарплата) - 0.0%


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


## Умножение матриц

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Не изменится.

**Обоснование:**

Для начала укажем основные свойства матриц:

$$(А(BC) = (AB)C)$$
$$(P^T)^{-1} = (P^{-1})^T$$
$$(AB)^T = B^TA^T$$
$$(AB)^{-1} = B^{-1}A^{-1}$$
$$AA^{-1} = A^{-1}A = E$$
$$AE = A$$

Заменим $w$ в формуле предсказания:
$$a = Xw = X(X^TX)^{-1}X^Ty$$

Перейдем к доказательству. Домножим матрицу признаков $X$ на обратимую матрицу $P$, выпишем формулу для новых предсказаний $a_p$ и попробуем сократить ее:

$$a_p = X_p(X_p^TX_p)^{-1}X_p^Ty$$
$$=XP((XP)^TXP)^{-1}(XP)^Ty$$
$$=XP(P^TX^TXP)^{-1}P^TX^Ty$$
$$=XPP^{-1}(X^TX)^{-1}(P^{-1})^TP^TX^Ty$$
$$=XE(X^TX)^{-1}EX^Ty$$
$$=X(X^TX)^{-1}X^Ty$$ 
$$= Xw = a$$

Получили значение, равное исходному значению предсксказаний $a$, следовательно:
$$a = a_p$$

**Определим, как связаны параметры $w$ линейной регрессии в исходной задаче и в преобразованной:**

Преобразованная задача обучения выглядит следующим образом:
$$
w = \arg\min_w MSE(XPw_p, y)
$$

Раскроем вектор $w_p$ и сравним с вектором $w$:

$$w_p = ((XP)^T XP)^{-1} (XP)^Ty$$
$$= (XP)^{-1}(XP^T)^{-1}y$$
$$= (XP)^{-1}y$$
$$= P^{-1}X^{-1}y$$

$$w = (X^TX)^{-1}X^Ty$$
$$= X^{-1}(X^T)^{-1}X^Ty$$
$$=X^{-1}y$$

Можем сделать вывод что $w_p = P^{-1}w$

В итоге получаем:
$$
w_p = \arg\min_w MSE(XPP^{-1}w, y)
$$
Следовательно:
$$
w_p = \arg\min_w MSE(Xw, y) = w
$$

Получили взаимно-однозначное соответствие параметров исходно и преобразованной матрицы, минимум будет достигаться при одном и том же значении функции потерь. 

## Алгоритм преобразования

**Алгоритм**

Для того, чтобы защитить данные пользователей предпримем следующие шаги:

- Создадим произвольную квадратную матрицу с размерностью равной количеству строк матрицы признаков.
- Проверим, что она обратима. Если нет, то надо создать новую матрицу, потому что иначе мы не сможем восстановить данные
- Перемножим матрицу признаков и полученную обратимую, тем самым зашифровывая истинные значения

**Обоснование**

На основании доказательства из предыдущего пункта можно заключить, что перемножение матриц не повлияет на качество линейной регрессии.

## Проверка алгоритма

Выделим признаки и целевой показатель `Страховые выплаты`

In [44]:
features = data.drop(columns=['Страховые выплаты'])
target = data['Страховые выплаты']

In [45]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=RANDOM_STATE
)

Создадим произвольную матрицу и проверим, что она обратима

In [46]:
def get_matrix(features):
    n = features.shape[1]
    random_matrix = np.random.randint(1, 10, (n,n))
    det = np.linalg.det(random_matrix)
    while det == 0:
        random_matrix = np.random.randint(1, 10, (n,n))
        det = np.linalg.det(random_matrix)
    return random_matrix

def transfom_features(features1, features2):
    display(features1.head(), features2.head())
    matrix = get_matrix(features)
    crypted_features1 = features1 @ matrix
    crypted_features2 = features2 @ matrix
    display(crypted_features1.head(), crypted_features2.head())
    display(matrix)
    print('Размеры обучающей и тестовой выборoк:',crypted_features1.shape, crypted_features2.shape)
    return crypted_features1, crypted_features2

In [47]:
features_train_crypted, features_test_crypted = transfom_features(features_train, features_test)

,Пол,Возраст,Зарплата,Члены семьи
3369,1,43.0,36200.0,1
1441,1,34.0,57600.0,0
571,0,32.0,41100.0,1
225,0,36.0,45100.0,1
2558,0,33.0,50600.0,2


,Пол,Возраст,Зарплата,Члены семьи
3183,0,33.0,39000.0,4
1071,0,50.0,43100.0,2
2640,1,39.0,42100.0,0
2282,0,20.0,34800.0,0
1595,0,41.0,40000.0,4


,0,1,2,3
3369,217260.0,72662.0,108823.0,144899.0
1441,345642.0,115407.0,172974.0,230475.0
571,246641.0,82393.0,123464.0,164470.0
225,270645.0,90417.0,135484.0,180478.0
2558,303651.0,101400.0,151973.0,202478.0


,0,1,2,3
3183,234069.0,78202.0,117181.0,156090.0
1071,258668.0,86502.0,129558.0,172512.0
2640,252647.0,84437.0,126499.0,168485.0
2282,208820.0,69720.0,104500.0,139240.0
1595,240077.0,80250.0,120221.0,160106.0


array([[8, 3, 4, 7],
       [1, 6, 5, 2],
       [6, 2, 3, 4],
       [9, 1, 4, 6]])

Размеры обучающей и тестовой выборoк: (3750, 4) (1250, 4)


Обучим модели линейной регрессии, на оригинальных данных, преобразованных, оригинальных данных(масштабированных), преобразованных(масштабированных).
После этого проверим, будет ли существенная разница.

In [50]:
model = LinearRegression()

model.fit(features_train, target_train)
predictions_original = model.predict(features_test)
r2_original = r2_score(target_test, predictions_original)
print('Метрика R2 для оригинальных признаков: {}'.format(r2_original))

Метрика R2 для оригинальных признаков: 0.43522757127026546


In [51]:
model.fit(features_train_crypted, target_train)
predictions_crypted = model.predict(features_test_crypted)
r2_crypted = r2_score(target_test, predictions_crypted)
print('Метрика R2 для преобразованных признаков: {}'.format(r2_crypted))

Метрика R2 для преобразованных признаков: 0.43522757127022094


In [52]:
scaler = MinMaxScaler()
pipeline = Pipeline([("standard_scaler", scaler),("linear_regression", model)])
pipeline.fit(features_train, target_train)
predictions_original_scaled = pipeline.predict(features_test)
r2_original_scaled = r2_score(target_test, predictions_original_scaled)
print('Метрика R2 для оригинальных(масштабированных) признаков: {}'.format(r2_original_scaled))

Метрика R2 для оригинальных(масштабированных) признаков: 0.4352275712702667


In [53]:
pipeline = Pipeline([("standard_scaler", scaler),("linear_regression", model)])
pipeline.fit(features_train_crypted, target_train)
predictions_crypted_scaled = pipeline.predict(features_test_crypted)
r2_crypted_scaled = r2_score(target_test, predictions_crypted_scaled)
print('Метрика R2 для преобразованных(масштабированных) признаков: {}'.format(r2_crypted_scaled))

Метрика R2 для преобразованных(масштабированных) признаков: 0.43522757127056333


In [54]:
result = pd.DataFrame(data= [r2_original,
                      r2_original_scaled,
                      r2_crypted,
                      r2_crypted_scaled], 
                     columns=['Коэффициент детерминации'], 
                     index=['Линейная регрессия',
                            'Линейная регрессия c масштабом',
                            'Линейная регрессия на преобразованных признаках',
                            'Линейная регрессия на преобразованных признаках c масштабом',])
result

,Коэффициент детерминации
Линейная регрессия,0.435228
Линейная регрессия c масштабом,0.435228
Линейная регрессия на преобразованных признаках,0.435228
Линейная регрессия на преобразованных признаках c масштабом,0.435228


**Вывод**

В ходе работы было проделано:

- Загружены и изучены данные.
- Качество линейной регресии при использовании исходной матрицы и исходной матрицы, умноженной на обратимую не изменяется.
- Создан алгоритм преобразования данных.
- Исследован алгоритм преобразования данных и проверен коэффициент детерминации для данных без преобразования и с ним.
- По результатам работы использования матричных операций можно увидеть, что данные очень просто зашифровать от распознования, имея правильную матрицу.

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования